<a id="item31"></a>


In [1]:
#libraries import
import pandas as pd
import numpy as np

In [2]:
#load data
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  Strength  
0            1040.0           676.0   28     79.99  
1            1055.0           676.0   28     61.89  
2             932.0           594.0  270     40.27  
3             932.0           594.0  365     41.05  
4             978.4           825.5  360     44.30

In [3]:
#number of data points
concrete_data.shape

(1030, 9)

In [4]:
#data basic stats
concrete_data.describe()

Cement  Blast Furnace Slag      Fly Ash        Water  \
count  1030.000000         1030.000000  1030.000000  1030.000000   
mean    281.167864           73.895825    54.188350   181.567282   
std     104.506364           86.279342    63.997004    21.354219   
min     102.000000            0.000000     0.000000   121.800000   
25%     192.375000            0.000000     0.000000   164.900000   
50%     272.900000           22.000000     0.000000   185.000000   
75%     350.000000          142.950000   118.300000   192.000000   
max     540.000000          359.400000   200.100000   247.000000   

       Superplasticizer  Coarse Aggregate  Fine Aggregate          Age  \
count       1030.000000       1030.000000     1030.000000  1030.000000   
mean           6.204660        972.918932      773.580485    45.662136   
std            5.973841         77.753954       80.175980    63.169912   
min            0.000000        801.000000      594.000000     1.000000   
25%            0.000000        932.000000      730.950000     7.000000   
50%            6.400000        968.000000      779.500000    28.000000   
75%           10.200000       1029.400000      824.000000    56.000000   
max           32.200000       1145.000000      992.600000   365.000000   

          Strength  
count  1030.000000  
mean     35.817961  
std      16.705742  
min       2.330000  
25%      23.710000  
50%      34.445000  
75%      46.135000  
max      82.600000

In [5]:
#check for nulls and if data is clean and ready to use
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   Coarse Aggregate  Fine Aggregate  Age  
0            1040.0           676.0   28  
1            1055.0           676.0   28  
2             932.0           594.0  270  
3             932.0           594.0  365  
4             978.4           825.5  360

In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [9]:
n_cols = predictors.shape[1] # number of predictors
print(n_cols)

8


<a id="item1"></a>


<a id='item32'></a>


## Import Keras


#### Let's go ahead and import the Keras library


In [10]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [11]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


In [13]:
from sklearn.model_selection import train_test_split

# Split the data
#x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.33, shuffle= True)

mse_list = []
#run for 50 times
for i in range(50):
    # build the model
    model = regression_model()
    # fit the model
    # verbose: 0 for no logging to stdout, 1 for progress bar logging, 2 for one log line per epoch
    # use shuffle to true to enable splitting the data
    history = model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=0, shuffle = True)
    mse_list.append(history.history['val_loss'][-1])

2021-10-08 14:52:02.983237: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-10-08 14:52:02.995437: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2095145000 Hz
2021-10-08 14:52:02.997256: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55aa25634090 executing computations on platform Host. Devices:
2021-10-08 14:52:02.997316: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-10-08 14:52:03.100255: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not 

In [14]:
for idx, val in enumerate(mse_list):
    print(idx, val)

0 185.78683918579497
1 1100.0085186510796
2 603.472658027723
3 335.76897551403846
4 92.88198741431376
5 239.41564338955678
6 341.54365385234547
7 290.0487139062974
8 124.90888873350274
9 81.77703611750432
10 293.24769918201036
11 474.89014076800794
12 360.6785819538203
13 612.9698678914783
14 1638.7136475399475
15 97.27990075762604
16 2928.9650685331967
17 171.92558076235082
18 106.55186722888143
19 109.20408936534498
20 223.85906194791824
21 425.08611902527053
22 375.56988703162926
23 940.217402782255
24 305.2156149854938
25 87.61243843646497
26 90.24338321624064
27 724.6189616675516
28 99.27090505951817
29 182.22252914280568
30 264.62418288937664
31 105.19228261805661
32 646.6243420449662
33 723.130874880695
34 1444.1734982586215
35 788.9913341929612
36 659.7862687095469
37 84.13544377694238
38 72.79005967914865
39 218.85765801587152
40 555.6398327281174
41 268.69729609319694
42 176.67303306307994
43 115.6365554710808
44 918.8702039008773
45 91.87329405256845
46 265.32760205315157
47

In [15]:
print("mean mse_list=", np.mean(mse_list), "std mse_list", np.std(mse_list))

mean mse_list= 428.2980590773401 std mse_list 501.07992890249454
